In [134]:
import findspark
findspark.init()

In [135]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark

**Employee Data Analysis** [Link to Dataset](https://www.kaggle.com/datasets/madhurpant/data-science-jobs-in-india)




In [136]:
df=spark.read.csv('../Data_Science_Jobs_in_India.csv',header=True)
df.show(10)

+---+-----------------+--------------+--------------+----------+----------+----------+---------------+
|_c0|     company_name|     job_title|min_experience|avg_salary|min_salary|max_salary|num_of_salaries|
+---+-----------------+--------------+--------------+----------+----------+----------+---------------+
|  0|              TCS|Data Scientist|             2|      7.8L|      4.5L|     16.0L|            841|
|  1|        Accenture|Data Scientist|             2|     12.8L|      5.8L|     23.0L|            501|
|  2|              IBM|Data Scientist|             2|     13.4L|      5.3L|     25.0L|            394|
|  3|        Cognizant|Data Scientist|             2|      9.8L|      5.0L|     18.0L|            318|
|  4|        Capgemini|Data Scientist|             2|      8.6L|      4.8L|     14.6L|            300|
|  5|          Infosys|Data Scientist|             2|      9.3L|      4.5L|     24.0L|            228|
|  6|            Wipro|Data Scientist|             2|      9.7L|      4.5

22/11/04 14:15:39 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , company_name, job_title, min_experience, avg_salary, min_salary, max_salary, num_of_salaries
 Schema: _c0, company_name, job_title, min_experience, avg_salary, min_salary, max_salary, num_of_salaries
Expected: _c0 but found: 
CSV file: file:///Users/adityabhat/Documents/Projects/SparkDemo/Data_Science_Jobs_in_India.csv


In [137]:
#Find if salary is purely in Lakhs

df.select('avg_salary')\
    .withColumn('Unit' ,regexp_replace(col('avg_salary'),'[0-9.]',''))\
    .filter(col('avg_salary').rlike('\d'))\
    .select("Unit").distinct().show()

#So all the salary is in Lakhs

+----+
|Unit|
+----+
|   L|
+----+



In [138]:
from pyspark.sql.functions import col,regexp_replace
salary_cols=['avg_salary','min_salary','max_salary']
df=df.drop('_c0')
for c in salary_cols:
    df=df.withColumn(c,regexp_replace(col(c),'L','').cast('double'))

df.show(5)

# df.selectExpr("max(cast(replace(avg_salary,'L','') as double)) as MaxOfSalary").show()

+------------+--------------+--------------+----------+----------+----------+---------------+
|company_name|     job_title|min_experience|avg_salary|min_salary|max_salary|num_of_salaries|
+------------+--------------+--------------+----------+----------+----------+---------------+
|         TCS|Data Scientist|             2|       7.8|       4.5|      16.0|            841|
|   Accenture|Data Scientist|             2|      12.8|       5.8|      23.0|            501|
|         IBM|Data Scientist|             2|      13.4|       5.3|      25.0|            394|
|   Cognizant|Data Scientist|             2|       9.8|       5.0|      18.0|            318|
|   Capgemini|Data Scientist|             2|       8.6|       4.8|      14.6|            300|
+------------+--------------+--------------+----------+----------+----------+---------------+
only showing top 5 rows



**Find the highest paying job (regardless of experience)**


In [139]:
max_sal=df.selectExpr("max(avg_salary)").collect()[0][0]
df.filter(col('avg_salary')==max_sal).show(truncate=False)

+------------+---------------------+--------------+----------+----------+----------+---------------+
|company_name|job_title            |min_experience|avg_salary|min_salary|max_salary|num_of_salaries|
+------------+---------------------+--------------+----------+----------+----------+---------------+
|Flipkart    |Senior Data Scientist|5             |82.0      |55.0      |100.0     |5              |
+------------+---------------------+--------------+----------+----------+----------+---------------+



**Calculate Avg Salary By Job titles.**


In [140]:
from pyspark.sql.functions import round
df.groupBy('job_title').avg('avg_salary')\
        .select('job_title',round(col('avg(avg_salary)'),2).alias('avg_salary'))\
        .show(truncate=False)

+-------------------------+----------+
|job_title                |avg_salary|
+-------------------------+----------+
|Machine Learning Engineer|9.85      |
|Business Analyst         |8.95      |
|Data Scientist           |13.53     |
|Data Analyst             |5.71      |
|Senior Business Analyst  |13.17     |
|Senior Data Analyst      |9.57      |
|Senior Data Scientist    |22.29     |
|Data Architect           |25.09     |
|Senior Data Engineer     |19.0      |
|Data Engineer            |11.81     |
+-------------------------+----------+



**Which Company pays highest for the least amount of experience ( group by Job Title order by salary).**

In [144]:
min_exp=df.selectExpr("min(min_experience)").collect()[0][0]
df_min_exp=df.filter(col('min_experience')==min_exp)
    
df_max_by_title=df_min_exp.groupBy('job_title').max('avg_salary')\
        .select(col('job_title').alias('designation'),col('max(avg_salary)').alias('max_of_avg_salary'))

df_max_by_title.show(truncate=False)

+-------------------------+-----------------+
|designation              |max_of_avg_salary|
+-------------------------+-----------------+
|Machine Learning Engineer|19.4             |
|Business Analyst         |14.9             |
|Data Scientist           |23.2             |
|Data Analyst             |10.4             |
|Senior Business Analyst  |13.0             |
|Senior Data Analyst      |7.7              |
|Senior Data Scientist    |24.6             |
|Senior Data Engineer     |11.8             |
|Data Engineer            |26.5             |
+-------------------------+-----------------+



In [147]:
from pyspark.sql.functions import broadcast,desc

df.join(broadcast(df_max_by_title),on=( (df_max_by_title.designation==df.job_title) &\
                    (df_max_by_title.max_of_avg_salary==df.avg_salary) ),how='right')\
    .filter(col('min_experience')==min_exp)\
    .select('job_title','company_name','min_experience','avg_salary')\
    .orderBy(desc('avg_salary'))\
    .show(truncate=False)




+-------------------------+----------------------+--------------+----------+
|job_title                |company_name          |min_experience|avg_salary|
+-------------------------+----------------------+--------------+----------+
|Data Engineer            |Autodesk              |0             |26.5      |
|Senior Data Scientist    |HSBC                  |0             |24.6      |
|Data Scientist           |Flipkart              |0             |23.2      |
|Machine Learning Engineer|Qualcomm              |0             |19.4      |
|Business Analyst         |Goldman Sachs         |0             |14.9      |
|Senior Business Analyst  |Unacademy             |0             |13.0      |
|Senior Data Engineer     |Wavicle Data Solutions|0             |11.8      |
|Data Analyst             |Citibank              |0             |10.4      |
|Senior Data Analyst      |Shadowfax Technologies|0             |7.7       |
+-------------------------+----------------------+--------------+----------+